In [1]:
from sklearn.svm import SVC
import sys
import os
import numpy as np
import cv2
from PIL import Image
from PIL.ImageQt import ImageQt
from PyQt5 import QtCore, QtGui, QtWidgets, uic
from PyQt5.QtWidgets import QWidget, QLabel
import pickle

In [2]:
def save_pickle(filename, data):
    with open(filename, "wb") as fo:
        pickle.dump(data, fo, protocol=pickle.HIGHEST_PROTOCOL)

def load_pickle(filename):
    with open(filename, 'rb') as fo:
        return pickle.load(fo)

# ---------------- Main App ----------------
class GaitDemo(QtWidgets.QMainWindow):
    def __init__(self):
        super().__init__()
        uic.loadUi("stridesentinelUI.ui", self)
        self.showFullScreen()

        os.makedirs("gei", exist_ok=True)

        # Camera
        self.capture = cv2.VideoCapture(0)
        if not self.capture.isOpened():
            print("Error: Could not open camera.")
            sys.exit()

        # Background subtractor
        self.bg_sub = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=40)

        # States
        self.register_state = False
        self.recognition_state = False
        self.save_on = False
        self.gei_fix_num = 20
        self.numInGEI = 0
        self.gei_current = np.zeros((128, 88), np.single)

        # Buttons
        self.save_gei.clicked.connect(self.save_gei_f)
        self.register_2.clicked.connect(self.register_show)
        self.recognize.clicked.connect(self.recognition_show)
        self.updater.clicked.connect(self.update_bk)

        # Data & model
        self.load_dataset()

        # Timer
        self._timer = QtCore.QTimer(self)
        self._timer.timeout.connect(self.play)
        self._timer.start(30)

        self.show()

    # ---------------- UI callbacks ----------------
    def save_gei_f(self):
        self.save_on = True
        self.state_print.setPlainText('Saving!')

    def register_show(self):
        self.register_state = True
        self.recognition_state = False
        self.state_print.setPlainText('Register!')
        self.gei_current = np.zeros((128, 88), np.single)
        self.numInGEI = 0

    def recognition_show(self):
        if self.num < 2 or self.model is None:
            self.state_print.setPlainText("Need at least 2 samples!")
            return
        self.recognition_state = True
        self.register_state = False
        self.state_print.setPlainText('Recognize!')
        self.gei_current = np.zeros((128, 88), np.single)
        self.numInGEI = 0

    # ---------------- Dataset & Model ----------------
    def load_dataset(self):
        self.data_path = './GaitData.pkl'
        if QtCore.QFile.exists(self.data_path):
            dic = load_pickle(self.data_path)
            self.num = dic['num']
            self.gei = dic['gei']
            self.name = dic['name']
        else:
            self.num = 0
            self.gei = np.zeros((0, 128, 88), np.uint8)
            self.name = []
            dic = {'num': self.num, 'gei': self.gei, 'name': self.name}
            save_pickle(self.data_path, dic)

        self.id_num.setPlainText('%d' % self.num)
        self.state_print.setPlainText('Running!')
        self.train_model()

    def train_model(self):
        self.model = None
        if self.num >= 2:
            X = self.gei[:self.num].reshape(self.num, -1)
            y = np.array(self.name)
            self.model = SVC(kernel='rbf', probability=True)
            self.model.fit(X, y)
            print("SVM trained with", self.num, "samples")

    # ---------------- Main loop ----------------
    def play(self):
        ret, frame = self.capture.read()
        if not ret:
            return

        frame = cv2.resize(frame, (512, 384))
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (5, 5), 0)

        # Background subtraction
        thresh = self.bg_sub.apply(gray)

        # Morphology cleanup
        kernel = np.ones((5, 5), np.uint8)
        thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
        thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

        cnts, _ = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        max_rec = 0
        x_max = y_max = w_max = h_max = 0

        for c in cnts:
            if cv2.contourArea(c) < 800:
                continue
            (x, y, w, h) = cv2.boundingRect(c)
            if w > 25 and h > 50 and w * h > max_rec:
                max_rec = w * h
                x_max, y_max, w_max, h_max = x, y, w, h

        if max_rec > 0:
            cv2.rectangle(frame, (x_max, y_max),
                          (x_max + w_max, y_max + h_max), (0, 255, 0), 2)

            if self.register_state or self.recognition_state:
                roi = thresh[y_max:y_max + h_max, x_max:x_max + w_max]
                if roi.size > 0:
                    roi = cv2.resize(roi, (88, 128))
                    if self.numInGEI < self.gei_fix_num:
                        self.gei_current += roi
                    self.numInGEI += 1

        # -------- After enough frames --------
        if self.numInGEI >= self.gei_fix_num:
            gei_img = (self.gei_current / self.gei_fix_num).astype(np.uint8)

            # -------- Save --------
            if self.save_on and self.register_state:

                # 🔧 FIX: expand array dynamically
                if self.num >= self.gei.shape[0]:
                    self.gei = np.vstack([self.gei,
                                          np.zeros((1, 128, 88), dtype=np.uint8)])

                self.gei[self.num, :, :] = gei_img
                Image.fromarray(gei_img).save(f'./gei/gei{self.num:02d}.jpg')
                self.name.append(self.id_name.toPlainText())
                self.num += 1

                dic = {'num': self.num, 'gei': self.gei[:self.num], 'name': self.name}
                save_pickle(self.data_path, dic)

                self.id_num.setPlainText('%d' % self.num)
                self.state_print.setPlainText('Saved!')
                self.save_on = False

                self.train_model()
                self.gei_current = np.zeros((128, 88), np.single)
                self.numInGEI = 0

            # -------- Recognize (SVM) --------
            elif self.recognition_state and self.model is not None:
                Xq = gei_img.reshape(1, -1)
                probs = self.model.predict_proba(Xq)[0]
                idx = np.argmax(probs)
                name_rec = self.model.classes_[idx]
                conf = probs[idx] * 100

                if conf < 60:
                    text = "Unknown"
                else:
                    text = f"{name_rec} ({conf:.1f}%)"

                cv2.putText(frame, text,
                            (x_max + 20, y_max + 20),
                            cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 2)

                self.gei_current = np.zeros((128, 88), np.single)
                self.numInGEI = 0

        # ---------------- Display ----------------
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        qimage = QtGui.QImage(rgb, rgb.shape[1], rgb.shape[0],
                              QtGui.QImage.Format_RGB888)
        self.video_label.setPixmap(QtGui.QPixmap.fromImage(qimage))

        seg = np.repeat(thresh[:, :, np.newaxis], 3, axis=2)
        seg[:, :, 0] = 0
        seg[:, :, 2] = 0
        qseg = QtGui.QImage(seg, seg.shape[1], seg.shape[0],
                            QtGui.QImage.Format_RGB888)
        self.seg_label.setPixmap(QtGui.QPixmap.fromImage(qseg))

    def update_bk(self):
        self.bg_sub = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=40)
        self.state_print.setPlainText("Background Updated")

    def closeEvent(self, event):
        self.capture.release()
        event.accept()

# ---------------- Run ----------------
if __name__ == "__main__":
    app = QtWidgets.QApplication(sys.argv)
    window = GaitDemo()
    app.exec_()   # for Jupyter; use sys.exit(app.exec_()) in CMD


SVM trained with 2 samples
SVM trained with 3 samples
